In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, read_pkl
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from joblib import delayed, Parallel

In [2]:
df_train = read_pkl('../input/train_df.pkl')


#==============================================================================
# PICKLE READ SUCCESS !!! ../input/train_df.pkl
#==============================================================================



In [3]:
df_test = read_pkl('../input/test_df.pkl')


#==============================================================================
# PICKLE READ SUCCESS !!! ../input/test_df.pkl
#==============================================================================



In [10]:
pd.set_option('max_columns', 800)
df_train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,DT,DT_M,DT_W,DT_D,DT_hour,DT_day_week,DT_day_month,is_december,is_holiday,DT_M_total,DT_W_total,DT_D_total,uid,uid2,uid3,uid4,uid5,card1_fq_enc,card2_fq_enc,card3_fq_enc,card5_fq_enc,uid_fq_enc,uid2_fq_enc,uid3_fq_enc,uid4_fq_enc,uid5_fq_enc,bank_type,card3_DT_D_hour_dist,card3_DT_W_week_day_dist,card3_DT_M_month_day_dist,card3_DT_D_hour_dist_best,card3_DT_W_week_day_dist_best,card3_DT_M_month_day_dist_best,card5_DT_D_hour_dist,card5_DT_W_week_day_dist,card5_DT_M_month_day_dist,card5_DT_D_hour_dist_best,card5_DT_W_week_day_dist_best,card5_DT_M_month_day_dist_best,bank_type_DT_D_hour_dist,bank_type_DT_W_week_day_dist,bank_type_DT_M_month_day_dist,bank_type_DT_D_hour_dist_best,bank_type_DT_W_week_day_dist_best,bank_type_DT_M_month_day_dist_best,bank_type_DT_M,bank_type_DT_W,bank_type_DT_D,uid_D1_mean,uid_D1_std,uid2_D1_mean,uid2_D1_std,uid3_D1_mean,uid3_D1_std,uid4_D1_mean,uid4_D1_std,uid5_D1_mean,uid5_D1_std,bank_type_D1_mean,bank_type_D1_std,uid_D2_mean,uid_D2_std,uid2_D2_mean,uid2_D2_std,uid3_D2_mean,uid3_D2_std,uid4_D2_mean,uid4_D2_std,uid5_D2_mean,uid5_D2_std,bank_type_D2_mean,bank_type_D2_std,uid_D3_mean,uid_D3_std,uid2_D3_mean,uid2_D3_std,uid3_D3_mean,uid3_D3_std,uid4_D3_mean,uid4_D3_std,uid5_D3_mean,uid5_D3_std,bank_type_D3_mean,bank_type_D3_std,uid_D4_mean,uid_D4_std,uid2_D4_mean,uid2_D4_std,uid3_D4_mean,uid3_D4_std,uid4_D4_mean,uid4_D4_std,uid5_D4_mean,uid5_D4_std,bank_type_D4_mean,bank_type_D4_std,uid_D5_mean,uid_D5_std,uid2_D5_mean,uid2_D5_std,uid3_D5_mean,uid3_D5_std,uid4_D5_mean,uid4_D5_std,uid5_D5_mean,uid5_D5_std,bank_type_D5_mean,bank_type_D5_std,uid_D6_mean,uid_D6_std,uid2_D6_mean,uid2_D6_std,uid3_D6_mean,uid3_D6_std,uid4_D6_mean,uid4_D6_std,uid5_D6_mean,uid5_D6_std,bank_type_D6_mean,bank_type_D6_std,uid_D7_mean,uid_D7_std,uid2_D7_mean,uid2_D7_std,uid3_D7_mean,uid3_D7_std,uid4_D7_mean,uid4_D7_std,uid5_D7_mean,uid5_D7_std,bank_type_D7_mean,bank_type_D7_std,uid_D8_mean,uid_D8_std,uid2_D8_mean,uid2_D8_std,uid3_D8_mean,uid3_D8_std,uid4_D8_mean,uid4_D8_std,uid5_D8_mean,uid5_D8_std,bank_type_D8_mean,bank_type_D8_std,uid_D9_mean,uid_D9_std,uid2_D9_mean,uid2_D9_std,uid3_D9_mean,uid3_D9_std,uid

In [5]:
dir_save = 'valid'
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD', 'DT', 'DT_M', 'DT_W', 'DT_D', 'DT_hour']
cols_save = [col for col in df_train if col not in COLUMNS_IGNORE and col.count('uid') and col.count('D') and col.count('std') ]

def parallel_save(train, test):
    save_feature(train,  'ker', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
    save_feature(test,  'ker', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)
    
Parallel(60)(
    [delayed(parallel_save)(
        df_train[[feature]], df_test[[feature]]
    ) for feature in cols_save]) 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]